In [1]:
import pandas as pd

In [2]:
clinical = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\clinical.cart.2020-02-13\\clinical.tsv', sep='\t')
clusters = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\heatmap\\sample_clusters_filtered_4_log2-zscore_ward.D2_euclidean_12_3reorder.csv', sep=',')

In [3]:
print(clusters.shape) # (340, 2)
display(clusters.head(5))
print()
print(clinical.shape) # (742, 161)
display(clinical.head(5))
print()

(742, 161)


,case_id,submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,created_datetime,days_to_birth,days_to_death,...,tumor_grade,tumor_largest_dimension_diameter,tumor_regression_grade,tumor_stage,updated_datetime.3,vascular_invasion_present,vascular_invasion_type,weiss_assessment_score,wilms_tumor_histologic_subtype,year_of_diagnosis
0,c6a74db2-cf97-4c7b-be8c-557ee27e1d43,TCGA-RC-A6M3,TCGA-LIHC,24,--,--,--,--,-8899,--,...,not reported,--,--,stage ii,2019-08-08T17:07:06.321879-05:00,--,--,--,--,2009
1,c6a74db2-cf97-4c7b-be8c-557ee27e1d43,TCGA-RC-A6M3,TCGA-LIHC,24,--,--,--,--,-8899,--,...,not reported,--,--,stage ii,2019-08-08T17:07:06.321879-05:00,--,--,--,--,2009
2,4c960eee-e4b5-499d-a596-238aa78745a4,TCGA-DD-AAE9,TCGA-LIHC,69,--,--,--,--,-25451,--,...,not reported,--,--,stage i,2019-08-08T17:06:22.550117-05:00,--,--,--,--,2012
3,4c960eee-e4b5-499d-a596-238aa78745a4,TCGA-DD-AAE9,TCGA-LIHC,69,--,--,--,--,-25451,--,...,not reported,--,--,stage i,2019-08-08T17:06:22.550117-05:00,--,--,--,--,2012
4,073e51f9-fb79-4a5f-9ba0-2c9af03c0b85,TCGA-5C-A9VG,TCGA-LIHC,58,--,--,--,--,-21273,--,...,not reported,--,--,stage ii,2019-08-08T17:04:55.218319-05:00,--,--,--,--,2013



(340, 2)


,Sample,Cluster
0,A9GX.01A,Cluster 01
1,A5MZ.01A,Cluster 01
2,A75E.01A,Cluster 01
3,A25V.01A,Cluster 01
4,X5262.01A,Cluster 01


In [4]:
def exact_info(item, integer = False):
    clusters[item] = ''
    for sample in clusters["Sample"]:
        info = set(clinical[clinical['submitter_id'].str.contains(sample[:-4][-4:])][item])
        if len(info) != 1 :
            raise ValueError('Multiple or no values found.')
        else:
            info = list(info)[0]
            if info == '--':
                info = None
            else:
                if integer:
                    info = int(info)
            clusters.loc[clusters["Sample"]==sample, item] = info

In [5]:
exact_info('days_to_death', integer = True)
clusters['days_to_death'] = clusters['days_to_death'].astype('Int64')

In [17]:
exact_info("vital_status")
exact_info("days_to_diagnosis")
exact_info("days_to_last_follow_up")

In [18]:
print(clusters.shape) # (340, 2)
display(clusters.head(5))
print()

(340, 6)


,Sample,Cluster,days_to_death,vital_status,days_to_diagnosis,days_to_last_follow_up
0,A9GX.01A,Cluster 01,NaN,Alive,None,2442
1,A5MZ.01A,Cluster 01,91,Dead,None,None
2,A75E.01A,Cluster 01,NaN,Alive,None,507
3,A25V.01A,Cluster 01,NaN,Alive,None,860
4,X5262.01A,Cluster 01,103,Dead,None,None


In [34]:
clusters[(clusters["vital_status"]=="Dead")&(clusters["days_to_last_follow_up"]!=None)]

,Sample,Cluster,days_to_death,vital_status,days_to_diagnosis,days_to_last_follow_up
1,A5MZ.01A,Cluster 01,91,Dead,None,None
4,X5262.01A,Cluster 01,103,Dead,None,None
5,A9CD.01A,Cluster 01,1386,Dead,None,None
6,A9GW.01A,Cluster 01,1271,Dead,None,None
7,A1EG.01A,Cluster 01,1372,Dead,None,None
8,A3I1.01A,Cluster 01,247,Dead,None,None
9,A7MD.01A,Cluster 01,52,Dead,None,None
15,A119.01A,Cluster 01,223,Dead,None,None
16,AADM.01A,Cluster 01,12,Dead,None,None
17,A39X.01A,Cluster 01,1694,Dead,None,None


In [19]:
clusters['days_to_death'].isna().sum()

220

In [20]:
clusters.groupby(["Cluster", "vital_status"]).count()

Sample  days_to_death  days_to_diagnosis  \
Cluster    vital_status                                             
Cluster 01 Alive            102              0                  0   
           Dead              56             56                  0   
           Not Reported       1              0                  0   
Cluster 02 Alive             71              0                  0   
           Dead              44             44                  0   
Cluster 03 Alive             46              0                  0   
           Dead              20             20                  0   

                         days_to_last_follow_up  
Cluster    vital_status                          
Cluster 01 Alive                             98  
           Dead                              11  
           Not Reported                       1  
Cluster 02 Alive                             69  
           Dead                               9  
Cluster 03 Alive                             46  
           Dead                               2

In [27]:
clusters[clusters["vital_status"]=="Not Reported"]

,Sample,Cluster,days_to_death,vital_status,days_to_diagnosis,days_to_last_follow_up
99,A11A.01A,Cluster 01,NaN,Not Reported,None,79


In [21]:
clusters[clusters["vital_status"]=='Dead']

,Sample,Cluster,days_to_death,vital_status,days_to_diagnosis,days_to_last_follow_up
1,A5MZ.01A,Cluster 01,91,Dead,None,None
4,X5262.01A,Cluster 01,103,Dead,None,None
5,A9CD.01A,Cluster 01,1386,Dead,None,None
6,A9GW.01A,Cluster 01,1271,Dead,None,None
7,A1EG.01A,Cluster 01,1372,Dead,None,None
8,A3I1.01A,Cluster 01,247,Dead,None,None
9,A7MD.01A,Cluster 01,52,Dead,None,None
15,A119.01A,Cluster 01,223,Dead,None,None
16,AADM.01A,Cluster 01,12,Dead,None,None
17,A39X.01A,Cluster 01,1694,Dead,None,None


In [28]:
from lifelines.estimation import KaplanMeierFitter
kmf = KaplanMeierFitter()

ModuleNotFoundError: No module named 'lifelines.estimation'

In [87]:
f = clusters.Cluster == "Cluster 01"
T = clusters[f]['time']
C = clusters[f]['delta']

kmf.fit(T, event_observed=C)

AttributeError: Cannot access callable attribute 'isna' of 'SeriesGroupBy' objects, try using the 'apply' method

In [ ]:
f2 = tongue.type==2
T2 = tongue[f2]['time']
C2 = tongue[f2]['delta']

ax = plt.subplot(111)

kmf.fit(T, event_observed=C, label=['Type 1 DNA'])
kmf.survival_function_.plot(ax=ax)
kmf.fit(T2, event_observed=C2, label=['Type 2 DNA'])
kmf.survival_function_.plot(ax=ax)

plt.title('Lifespans of different tumor DNA profile')

kmf2 = plt.gcf()

In [ ]:
pyplot(kmf2, ci=False)